<!--BOOK_INFORMATION-->
<img style="float: right; width: 100px" src="https://raw.github.com/pyomeca/design/master/logo/logo_cropped.svg?sanitize=true">

# Pyosim in the cloud
## with [pyomeca](https://github.com/pyomeca/pyomeca)
Romain Martinez (martinez.staps@gmail.com | [GitHub](https://github.com/romainmartinez))


<!--NAVIGATION-->
< [Verification](01.00-verification.ipynb) | [Contents](Index.ipynb) | [Moment arm](01.02-moment-arm.ipynb) >

# Inverse kinematics

In [1]:
from pathlib import Path

import yaml

import numpy as np
import pandas as pd
from pyosim import Conf
from pyomeca import Analogs3d

import altair as alt

alt.data_transformers.enable("json")

DataTransformerRegistry.enable('json')

In [2]:
MODEL = "wu"
DATA = '1_inverse_kinematic'
EXT = 'mot'

In [3]:
aws_conf = yaml.safe_load(open("../conf.yml"))

conf = Conf(project_path=aws_conf["path"]["project"]["local"])
participants = conf.get_participants_to_process()
conf.check_confs()

Project loaded
Configuration file loaded


In [5]:
data = []
for iparticipant in participants:
    data_path = conf.project_path / iparticipant / DATA
    for ifile in data_path.glob(f"*.{EXT}"):
        f = ifile.stem.replace(f"{MODEL}_", "")
        meta = {
            "filename": ifile.stem,
            "participant": f[:4].lower(),
            "sex": f[4],
            "height": f[-3],
            "weight": f[5:].split("H")[0],
            "trial": f[-1],
        }
        d = Analogs3d.from_sto(ifile)
        data.append(
#             d.low_pass(freq=d.get_rate, order=2, cutoff=6)
            d
            .time_normalization()
            .update_misc(meta)
            .to_dataframe(add_metadata=["misc"])
        )
data = pd.concat(data).reset_index()

print(data.shape)
data.head()

(30603, 32)


,index,thorax_tilt,thorax_list,thorax_rotation,thorax_tx,thorax_ty,thorax_tz,sternoclavicular_r1,sternoclavicular_r2,sternoclavicular_r3,...,box_rotZ,box_transX,box_transY,box_transZ,filename,participant,sex,height,weight,trial
0,0,-42.592644,-14.145737,7.572943,-0.487607,0.563652,0.100781,-11.389635,-14.896497,0.0,...,19.962831,0.175948,0.031156,0.235846,wu_SarCF12H2_2,sarc,F,2,12,2
1,1,-43.147437,-14.164911,7.183366,-0.484973,0.561490,0.099862,-11.566313,-14.472965,0.0,...,19.963181,0.176046,0.031256,0.235880,wu_SarCF12H2_2,sarc,F,2,12,2
2,2,-43.647301,-14.134783,6.879335,-0.483162,0.559728,0.099355,-11.690381,-14.114090,0.0,...,19.963576,0.176111,0.031319,0.235921,wu_SarCF12H2_2,sarc,F,2,12,2
3,3,-44.166730,-14.088121,6.671722,-0.482100,0.558288,0.099047,-11.931346,-13.834145,0.0,...,19.967721,0.176293,0.031375,0.236032,wu_SarCF12H2_2,sarc,F,2,12,2
4,4,-44.645587,-14.027500,6.397621,-0.481363,0.557173,0.098904,-12.498474,-13.869905,0.0,...,19.985076,0.176525,0.031392,0.236212,wu_SarCF12H2_2,sarc,F,2,12,2


In [6]:
data_wide = data.melt(id_vars=["index", "filename", "participant", "sex", "height", "weight", "trial"])
data_wide.head()

,index,filename,participant,sex,height,weight,trial,variable,value
0,0,wu_SarCF12H2_2,sarc,F,2,12,2,thorax_tilt,-42.592644
1,1,wu_SarCF12H2_2,sarc,F,2,12,2,thorax_tilt,-43.147437
2,2,wu_SarCF12H2_2,sarc,F,2,12,2,thorax_tilt,-43.647301
3,3,wu_SarCF12H2_2,sarc,F,2,12,2,thorax_tilt,-44.166730
4,4,wu_SarCF12H2_2,sarc,F,2,12,2,thorax_tilt,-44.645587


<!--NAVIGATION-->
< [Verification](01.00-verification.ipynb) | [Contents](Index.ipynb) | [Moment arm](01.02-moment-arm.ipynb) >